In [1]:
import spacy
import numpy as np
import pandas as pd
import re
import nltk
import pandas as pd
import spacy
from cleantext import clean
from multiprocessing import Pool

# Loading the spaCy model
nlp = spacy.load('en_core_web_sm')

# Ensure that NLTK components are downloaded
nltk.download('popular')


Since the GPL-licensed package `unidecode` is not installed, using Python's `unicodedata` package which yields worse results.
[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     C:\Users\govin\AppData\Roaming\nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     C:\Users\govin\AppData\Roaming\nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     C:\Users\govin\AppData\Roaming\nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     C:\Users\govin\AppData\Roaming\nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     C:\Users\govin\AppData\Roaming\nltk_data...
[n

True

In [2]:
file_path = r"C:\Users\govin\Desktop\mimic-iv-note-deidentified-free-text-clinical-notes-2.2\mimic-iv-note-deidentified-free-text-clinical-notes-2.2\note\sample_data.csv"


In [3]:
df = pd.read_csv(file_path)
df

,Unnamed: 0,note_id,subject_id,hadm_id,note_type,note_seq,charttime,storetime,text
0,1,10000032-DS-21,10000032,22595853,DS,21,2180-05-07,2180-05-09 15:26:00,\r\nName: ___ Unit No: ...
1,2,10000032-DS-22,10000032,22841357,DS,22,2180-06-27,2180-07-01 10:15:00,\r\nName: ___ Unit No: ...
2,3,10000032-DS-23,10000032,29079034,DS,23,2180-07-25,2180-07-25 21:42:00,\r\nName: ___ Unit No: ...
3,4,10000032-DS-24,10000032,25742920,DS,24,2180-08-07,2180-08-10 05:43:00,\r\nName: ___ Unit No: ...


In [4]:
df = df[['subject_id', 'text']]


In [5]:
def parse_text(text):
    # Split the text into parts based on the identified headings
    import re
    pattern = re.compile(r'(Name|Unit No|Admission Date|Discharge Date|Date of Birth|Sex|Service|Allergies|Attending|Chief Complaint|Major Surgical or Invasive Procedure|History of Present Illness|Past Medical History|Social History|Family History|Physical Exam|Pertinent Results|Micro Data|Imaging Results|Brief Hospital Course|Medications on Admission|Discharge Medications|Discharge Disposition|Discharge Diagnosis|Discharge Condition|Discharge Instructions|Followup Instructions):')
    parts = pattern.split(text)[1:]  # Split and remove the first split since it's before the first heading

    # Create a dictionary to hold the extracted data
    data_dict = {}
    for i in range(0, len(parts)-1, 2):
        key = parts[i].strip()
        value = parts[i+1].strip()
        data_dict[key] = value

    return pd.Series(data_dict)

# Apply the function to the 'text' column
parsed_columns = df['text'].apply(parse_text)

# Concatenate the parsed columns with any other existing DataFrame columns
df_final = pd.concat([df, parsed_columns], axis=1).drop('text', axis=1)  # Optionally drop the original text column

df_final

,subject_id,Name,Unit No,Admission Date,Discharge Date,Date of Birth,Sex,Service,Allergies,Attending,...,Physical Exam,Pertinent Results,Brief Hospital Course,Medications on Admission,Discharge Medications,Discharge Disposition,Discharge Diagnosis,Discharge Condition,Discharge Instructions,Followup Instructions
0,10000032,___,___,___,___,___,F,MEDICINE,No Known Allergies / Adverse Drug Reactions,___,...,VS: 98.1 107/61 78 18 97RA \r\nGeneral: in NA...,___ 10:25PM GLUCOSE-109* UREA N-25* CREAT-0....,"___ HCV cirrhosis c/b ascites, hiv on ART, h/o...",The Preadmission Medication list is accurate a...,1. Albuterol Inhaler 2 PUFF IH Q4H:PRN wheezin...,Home,Ascites from Portal HTN,Mental Status: Clear and coherent.\r\nLevel of...,"Dear Ms. ___,\r\nIt was a pleasure taking care...",___
1,10000032,___,___,___,___,___,F,MEDICINE,Percocet,___.,...,ADMISSION PHYSICAL EXAM: \r\nVS: T98.1 105/57...,LABS ON ADMISSION:\r\n___ 04:10PM BLOOD ___ \r...,"___ with HIV on HAART, HCV cirrhosis with asci...",The Preadmission Medication list is accurate a...,"1. Acetaminophen 500 mg PO Q6H:PRN pain,fever ...",Home,Primary: diuretic refractory ascites\r\nSecon...,Mental Status: Clear and coherent.\r\nLevel of...,"Dear ___,\r\n\r\n___ was a pleasure to take ca...",___
2,10000032,___,___,___,___,___,F,MEDICINE,Percocet / Vicodin,___,...,ADMISSION PHYSICAL EXAM:\r\nVitals T: 98.7 BP:...,ADMISSION LABS:\r\n=================\r\n___ 06...,NaN,The Preadmission Medication list is accurate a...,1. Calcium Carbonate 500 mg PO BID \r\n2. Emtr...,Home,Primary:\r\nHypotension\r\nHyperkalemia\r\nAcu...,Mental Status: Clear and coherent.\r\nLevel of...,"Dear Ms. ___,\r\n\r\nYou were admitted to ___ ...",___
3,10000032,___,___,___,___,___,F,MEDICINE,Percocet / Vicodin,___.,...,ADMISSION PHYSICAL EXAM\r\n===================...,ADMISSION LABS\r\n==============\r\n___ 09:20P...,NaN,NaN,1. albuterol sulfate 90 mcg/actuation inhalati...,Home With Service\r\n \r\nFacility:\r\n___,PRIMARY DIAGNOSIS\r\n=================\r\n-Dec...,Mental Status: Clear and coherent.\r\nLevel of...,"Dear Ms. ___,\r\n\r\nIt was a pleasure caring ...",___


In [6]:
df_final.drop(df_final.columns[1:6], axis=1, inplace=True)


In [7]:
def merge_rows_by_subject_id(file_path):
    # Load the CSV file into a DataFrame
    df = pd.read_csv(file_path)
    
    # Define a custom aggregation function that concatenates strings with two newlines between them
    def custom_concat(series):
        return '\n\n'.join(series.astype(str))
    
    # Create a dictionary for aggregation where every column except 'subject_id' uses the custom_concat
    aggregation_functions = {col: custom_concat for col in df.columns if col != 'subject_id'}
    
    # Group by 'subject_id' and aggregate using the defined functions
    grouped_df = df.groupby('subject_id').agg(aggregation_functions).reset_index()
    
    return grouped_df

# Example usage
input_file_path = 'C:/Users/govin/Desktop/mimic-iv-note-deidentified-free-text-clinical-notes-2.2/mimic-iv-note-deidentified-free-text-clinical-notes-2.2/note/my_dataframe.csv'
merged_df = merge_rows_by_subject_id(input_file_path)

# Saving the result to a new CSV
output_file_path = 'C:/Users/govin/Desktop/mimic-iv-note-deidentified-free-text-clinical-notes-2.2/mimic-iv-note-deidentified-free-text-clinical-notes-2.2/note/merged_output.csv'
merged_df.to_csv(output_file_path, index=False)

print("Merged DataFrame has been saved to:", output_file_path)

Merged DataFrame has been saved to: C:/Users/govin/Desktop/mimic-iv-note-deidentified-free-text-clinical-notes-2.2/mimic-iv-note-deidentified-free-text-clinical-notes-2.2/note/merged_output.csv


In [8]:
def merge_rows_by_subject_id(file_path):
    # Load the CSV file into a DataFrame
    df = pd.read_csv(file_path)
    
    # Define a custom aggregation function that concatenates strings with two newlines between them
    def custom_concat(series):
        return '\n\n'.join(series.astype(str))
    
    # Define the columns to take the last entry only
    last_entry_columns = ['Service', 'Sex', 'Family History', 'Allergies','Past Medical History',"Discharge Disposition"]
    
    # Create a dictionary for aggregation
    # Use 'last' for specific columns and 'custom_concat' for others
    aggregation_functions = {col: (lambda x: x.iloc[-1]) if col in last_entry_columns else custom_concat for col in df.columns if col != 'subject_id'}
    
    # Group by 'subject_id' and aggregate using the defined functions
    grouped_df = df.groupby('subject_id').agg(aggregation_functions).reset_index()
    
    return grouped_df



# Example usage
input_file_path = 'C:/Users/govin/Desktop/mimic-iv-note-deidentified-free-text-clinical-notes-2.2/mimic-iv-note-deidentified-free-text-clinical-notes-2.2/note/my_dataframe.csv'
merged_df = merge_rows_by_subject_id(input_file_path)

# Saving the result to a new CSV
output_file_path = 'C:/Users/govin/Desktop/mimic-iv-note-deidentified-free-text-clinical-notes-2.2/mimic-iv-note-deidentified-free-text-clinical-notes-2.2/note/merged_output.csv'
merged_df.to_csv(output_file_path, index=False)

print("Merged DataFrame has been saved to:", output_file_path)

Merged DataFrame has been saved to: C:/Users/govin/Desktop/mimic-iv-note-deidentified-free-text-clinical-notes-2.2/mimic-iv-note-deidentified-free-text-clinical-notes-2.2/note/merged_output.csv


In [9]:
merged_df

,subject_id,Sex,Service,Allergies,Attending,Chief Complaint,Major Surgical or Invasive Procedure,History of Present Illness,Past Medical History,Social History,...,Physical Exam,Pertinent Results,Brief Hospital Course,Medications on Admission,Discharge Medications,Discharge Disposition,Discharge Diagnosis,Discharge Condition,Discharge Instructions,Followup Instructions
0,10000032,F,MEDICINE,Percocet / Vicodin,___\n\n___.\n\n___\n\n___.,Worsening ABD distension and pain\n\nabdominal...,Paracentesis\n\n___ diagnostic paracentesis\r\...,"___ HCV cirrhosis c/b ascites, hiv on ART, h/o...",- HCV Cirrhosis: genotype 3a \r\n- HIV: on HA...,___\n\n___\n\n___\n\n___,...,VS: 98.1 107/61 78 18 97RA \r\nGeneral: in NA...,___ 10:25PM GLUCOSE-109* UREA N-25* CREAT-0....,"___ HCV cirrhosis c/b ascites, hiv on ART, h/o...",The Preadmission Medication list is accurate a...,1. Albuterol Inhaler 2 PUFF IH Q4H:PRN wheezin...,Home With Service\r\n \r\nFacility:\r\n___,Ascites from Portal HTN\n\nPrimary: diuretic ...,Mental Status: Clear and coherent.\r\nLevel of...,"Dear Ms. ___,\r\nIt was a pleasure taking care...",___\n\n___\n\n___\n\n___


In [10]:
# Assuming merged_df is your DataFrame
def clean_text(text):
    """Clean placeholder text and standardize spacing."""
    if pd.isnull(text):
        return text  # Return NaN as is
    text = re.sub(r'___+', 'REDACTED', text)  # Replace placeholders with 'REDACTED'
    text = re.sub(r'\s+', ' ', text).strip()  # Reduce multiple spaces to one and strip leading/trailing spaces
    return text

# Apply the cleaning function to each column in the DataFrame
for column in merged_df.columns:
    if merged_df[column].dtype == object:  # Ensure the column is of type object (textual data)
        merged_df[column] = merged_df[column].apply(clean_text)



In [11]:
merged_df

,subject_id,Sex,Service,Allergies,Attending,Chief Complaint,Major Surgical or Invasive Procedure,History of Present Illness,Past Medical History,Social History,...,Physical Exam,Pertinent Results,Brief Hospital Course,Medications on Admission,Discharge Medications,Discharge Disposition,Discharge Diagnosis,Discharge Condition,Discharge Instructions,Followup Instructions
0,10000032,F,MEDICINE,Percocet / Vicodin,REDACTED REDACTED. REDACTED REDACTED.,Worsening ABD distension and pain abdominal fu...,Paracentesis REDACTED diagnostic paracentesis ...,"REDACTED HCV cirrhosis c/b ascites, hiv on ART...","- HCV Cirrhosis: genotype 3a - HIV: on HAART, ...",REDACTED REDACTED REDACTED REDACTED,...,VS: 98.1 107/61 78 18 97RA General: in NAD HEE...,REDACTED 10:25PM GLUCOSE-109* UREA N-25* CREAT...,"REDACTED HCV cirrhosis c/b ascites, hiv on ART...",The Preadmission Medication list is accurate a...,1. Albuterol Inhaler 2 PUFF IH Q4H:PRN wheezin...,Home With Service Facility: REDACTED,Ascites from Portal HTN Primary: diuretic refr...,Mental Status: Clear and coherent. Level of Co...,"Dear Ms. REDACTED, It was a pleasure taking ca...",REDACTED REDACTED REDACTED REDACTED


In [12]:
import os

# Get the current working directory
current_directory = os.getcwd()

print("The current working directory is:", current_directory)


The current working directory is: c:\Users\govin\AppData\Local\Programs\Microsoft VS Code


In [15]:
merged_df

,subject_id,Sex,Service,Allergies,Attending,Chief Complaint,Major Surgical or Invasive Procedure,History of Present Illness,Past Medical History,Social History,...,Physical Exam,Pertinent Results,Brief Hospital Course,Medications on Admission,Discharge Medications,Discharge Disposition,Discharge Diagnosis,Discharge Condition,Discharge Instructions,Followup Instructions
0,10000032,F,MEDICINE,Percocet / Vicodin,REDACTED REDACTED. REDACTED REDACTED.,Worsening ABD distension and pain abdominal fu...,Paracentesis REDACTED diagnostic paracentesis ...,"REDACTED HCV cirrhosis c/b ascites, hiv on ART...","- HCV Cirrhosis: genotype 3a - HIV: on HAART, ...",REDACTED REDACTED REDACTED REDACTED,...,VS: 98.1 107/61 78 18 97RA General: in NAD HEE...,REDACTED 10:25PM GLUCOSE-109* UREA N-25* CREAT...,"REDACTED HCV cirrhosis c/b ascites, hiv on ART...",The Preadmission Medication list is accurate a...,1. Albuterol Inhaler 2 PUFF IH Q4H:PRN wheezin...,Home With Service Facility: REDACTED,Ascites from Portal HTN Primary: diuretic refr...,Mental Status: Clear and coherent. Level of Co...,"Dear Ms. REDACTED, It was a pleasure taking ca...",REDACTED REDACTED REDACTED REDACTED


In [16]:
from transformers import pipeline

# Load the summarization model from Hugging Face's Transformers library
summarizer = pipeline("summarization", model="Falconsai/medical_summarization")

def summarize_text(text):
    """ Summarize the text using the loaded model, handling long texts by summarizing in parts if needed. """
    if len(text.strip()) == 0:
        return ""
    
    try:
        # Assuming the model can handle the entire text directly if not too long
        summary = summarizer(text, max_length=1024, min_length=150, do_sample=False)
        return summary[0]['summary_text']
    except Exception as e:
        print(f"Error summarizing text: {e}")
        return text

# List of columns to summarize
columns_to_summarize = [
    'Major Surgical or Invasive Procedure',
    'History of Present Illness',
    'Family History', 'Pertinent Results', 'Brief Hospital Course',
    'Discharge Diagnosis', 'Discharge Condition', 'Discharge Instructions'
]


# Apply summarization to each specified column and update the DataFrame directly
for column in columns_to_summarize:
    print(f"Summarizing {column}...")
    if merged_df[column].dtype == object:  # Ensure the column is of type object (textual data)
        merged_df[column] = merged_df[column].apply(lambda x: summarize_text(x) if pd.notna(x) else x)

# Save the modified DataFrame to a new CSV file, preserving all original rows and columns
merged_df.to_csv('summary_df1.csv', index=False)


C:\Users\govin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Your max_length is set to 1024, but your input_length is only 35. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=17)


Summarizing Major Surgical or Invasive Procedure...


Token indices sequence length is longer than the specified maximum sequence length for this model (1684 > 512). Running this sequence through the model will result in indexing errors


Summarizing History of Present Illness...


Your max_length is set to 1024, but your input_length is only 55. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=27)


...

Summarizing Discharge Diagnosis...


Your max_length is set to 1024, but your input_length is only 115. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=57)


Summarizing Discharge Condition...


Your max_length is set to 1024, but your input_length is only 737. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=368)


Summarizing Discharge Instructions...


In [17]:
merged_df.to_csv('summary_df1.csv', index=False)


In [18]:
summ=pd.read_csv("summary_df1.csv")

In [19]:
summ

,subject_id,Sex,Service,Allergies,Attending,Chief Complaint,Major Surgical or Invasive Procedure,History of Present Illness,Past Medical History,Social History,...,Physical Exam,Pertinent Results,Brief Hospital Course,Medications on Admission,Discharge Medications,Discharge Disposition,Discharge Diagnosis,Discharge Condition,Discharge Instructions,Followup Instructions
0,10000032,F,MEDICINE,Percocet / Vicodin,REDACTED REDACTED. REDACTED REDACTED.,Worsening ABD distension and pain abdominal fu...,"paracentesis is a rare , non - invasive , inva...","REDACTED HCV cirrhosis c/b ascites, hiv on ART...","- HCV Cirrhosis: genotype 3a - HIV: on HAART, ...",REDACTED REDACTED REDACTED REDACTED,...,VS: 98.1 107/61 78 18 97RA General: in NAD HEE...,hepatic encephalopathy is a common comorbidity...,"REDACTED HCV cirrhosis c/b ascites, hiv on ART...",The Preadmission Medication list is accurate a...,1. Albuterol Inhaler 2 PUFF IH Q4H:PRN wheezin...,Home With Service Facility: REDACTED,ascites from portal hTN primary: diuretic refr...,background and objectives of this study are di...,a paracentesis was performed to remove 1.5L of...,REDACTED REDACTED REDACTED REDACTED


In [20]:
summ=summ.drop(["Attending","Followup Instructions","Social History"],axis=1)

In [34]:
summ

,subject_id,Sex,Service,Allergies,Chief Complaint,Major Surgical or Invasive Procedure,History of Present Illness,Past Medical History,Family History,Physical Exam,Pertinent Results,Brief Hospital Course,Medications on Admission,Discharge Medications,Discharge Disposition,Discharge Diagnosis,Discharge Condition,Discharge Instructions
0,10000032,F,MEDICINE,Percocet / Vicodin,Worsening ABD distension and pain abdominal fu...,"paracentesis is a rare , non - invasive , inva...","REDACTED HCV cirrhosis c/b ascites, hiv on ART...","- HCV Cirrhosis: genotype 3a - HIV: on HAART, ...","she is a total of five siblings , but she is n...",VS: 98.1 107/61 78 18 97RA General: in NAD HEE...,hepatic encephalopathy is a common comorbidity...,"REDACTED HCV cirrhosis c/b ascites, hiv on ART...",The Preadmission Medication list is accurate a...,1. Albuterol Inhaler 2 PUFF IH Q4H:PRN wheezin...,Home With Service Facility: REDACTED,ascites from portal hTN primary: diuretic refr...,background and objectives of this study are di...,a paracentesis was performed to remove 1.5L of...


In [33]:
summ.to_csv('summary_df1.csv', index=False)